# Diccionarios

## Acceso

In [ ]:
datos = {"nombre": "Ana", "puntos": 50}

# .keys() - Ver qué atributos tenemos
print(f"Claves disponibles: {list(datos.keys())}")

# .get() - Evitando errores
# Intentamos buscar 'email'. Como no existe, definimos que devuelva "No especificado"
email = datos.get("email", "No especificado")
print(f"Email: {email}")

## Métodos de Modificación (Escritura y Fusión)

In [ ]:
perfil = {"usuario": "admin", "tema": "claro"}
nuevas_configuraciones = {"tema": "oscuro", "idioma": "ES"}

# .update() - Fusionar configuraciones
# Nota cómo 'tema' cambia a 'oscuro' y se añade 'idioma'
perfil.update(nuevas_configuraciones)
print(f"Perfil actualizado: {perfil}")

# .setdefault() - Inicialización perezosa
# Intentamos fijar el puerto en 8080.
# Como 'puerto' no existe, lo crea.
puerto = perfil.setdefault("puerto", 8080)

# Intentamos fijar 'usuario' en 'invitado'.
# Como 'usuario' YA existe (es 'admin'), NO lo cambia y devuelve 'admin'.
user = perfil.setdefault("usuario", "invitado")

print(f"Perfil final: {perfil}")

## Métodos de Eliminación (Borrado)

In [ ]:
def gestionar_carrito():
    print("--- SISTEMA DE CARRITO DE COMPRAS ---\n")

    # 1. Carrito inicial
    carrito = {
        "Laptop": 1000,
        "Ratón": 20,
        "Teclado": 50
    }
    print(f"1. Carrito inicial: {carrito}")

    # 2. .update() -> Añadir un pack de productos de golpe
    promocion = {"Monitor": 200, "HDMI": 10, "Ratón": 15} # Nota: Ratón baja de precio
    carrito.update(promocion)
    print(f"2. Tras aplicar promoción (update): {carrito}")
    # Fíjate que 'Ratón' ahora vale 15 (sobrescrito) y Monitor/HDMI se añadieron.

    # 3. .get() -> Consultar precios sin miedo
    producto_buscado = "Impresora"
    precio = carrito.get(producto_buscado, "No disponible")
    print(f"3. Consultando '{producto_buscado}': {precio}")

    # 4. .setdefault() -> Aplicar cupón de descuento si no existe
    # Queremos asegurar que haya una clave 'descuento'.
    # Si ya hubiera uno, no lo tocamos.
    valor_descuento = carrito.setdefault("Descuento_Global", 0)
    print(f"4. Descuento aplicado: {valor_descuento}%")

    # 5. .pop() -> El usuario decide sacar un producto
    # Queremos quitar el 'Teclado' y saber cuánto costaba para restarlo del total visual
    producto_eliminado = "Teclado"
    # Usamos un valor default (0) por si el producto ya no estaba
    precio_eliminado = carrito.pop(producto_eliminado, 0)
    print(f"5. Eliminando {producto_eliminado}. Costaba: ${precio_eliminado}")

    # 6. .popitem() -> Deshacer la última acción (LIFO)
    # Imaginemos que el usuario dice "Quita lo último que se añadió"
    ultimo_item, ultimo_precio = carrito.popitem()
    print(f"6. Eliminando el último item añadido: {ultimo_item} (${ultimo_precio})")

    # 7. Cálculo final usando .values()
    # Ojo: 'Descuento_Global' está en el diccionario, hay que filtrarlo porque no es un precio sumable
    # Usamos dictionary comprehension para filtrar solo números que son precios de productos
    # (Asumimos que las claves de productos no empiezan por "Descuento")
    total = sum([v for k, v in carrito.items() if not k.startswith("Descuento")])

    print(f"\n--- ESTADO FINAL ---")
    print(f"Items: {list(carrito.keys())}")
    print(f"Total a pagar: ${total}")

    # 8. .clear() -> Finalizar compra
    carrito.clear()
    print(f"Compra finalizada. Carrito: {carrito}")

if __name__ == "__main__":
    gestionar_carrito()

# Procesos

In [ ]:
import multiprocessing
import time

def corredor(nombre):
    print(f"🏃 {nombre} ha salido!")
    # Simulamos que correr tarda 3 segundos
    time.sleep(3)
    print(f"🏁 {nombre} ha llegado a la meta.")

if __name__ == "__main__":
    print("--- INICIO DE LA CARRERA ---")
    start_time = time.time()

    # 1. Preparamos a los corredores (Procesos)
    # target = qué función ejecutar
    # args = qué nombre pasarle a la función (nota la coma extra: ("Nombre",))
    p1 = multiprocessing.Process(target=corredor, args=("Corredor A",))
    p2 = multiprocessing.Process(target=corredor, args=("Corredor B",))

    # 2. Disparo de salida (Empiezan a la vez)
    p1.start()
    p2.start()

    # 3. Esperamos a que ambos terminen
    # Si no ponemos esto, el programa principal termina y nos deja los tiempos mal
    p1.join()
    p2.join()

    end_time = time.time()
    print("--- FIN DE LA CARRERA ---")
    print(f"Tiempo total transcurrido: {end_time - start_time:.2f} segundos")

# Colas

In [ ]:
import multiprocessing
import time

def cocinero(ventanilla):
    """Produce datos y los pone en la cola"""
    menu = ["Hamburguesa", "Patatas Fritas", "Refresco"]

    for plato in menu:
        print(f"👨‍🍳 Cocinero: Preparando {plato}...")
        time.sleep(1) # Tiempo de cocinado

        # 1. PUT: Ponemos el dato en la cola
        ventanilla.put(plato)
        print(f"🔔 Cocinero: {plato} puesto en la ventanilla.")

def camarero(ventanilla):
    """Consume datos sacándolos de la cola"""
    # Sabemos que vienen 3 platos
    for i in range(3):
        print("💁 Camarero: Esperando en la ventanilla...")

        # 2. GET: Sacamos el dato.
        # IMPORTANTE: Si la cola está vacía, el camarero se queda QUIETO (bloqueado)
        # esperando hasta que aparezca algo. No gasta CPU esperando.
        plato = ventanilla.get()

        print(f"🚀 Camarero: Llevando {plato} a la mesa.\n")

if __name__ == "__main__":
    print("--- INICIO SERVICIO ---")

    # Creamos la Cola (es como un cable que conecta los procesos)
    cola_restaurante = multiprocessing.Queue()

    # Pasamos la MISMA cola a ambos procesos
    p1 = multiprocessing.Process(target=cocinero, args=(cola_restaurante,))
    p2 = multiprocessing.Process(target=camarero, args=(cola_restaurante,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print("--- FIN SERVICIO ---")

# Pipes

In [ ]:
import multiprocessing

def proceso_calculadora(conexion):
    """
    Este proceso vive al otro lado del tubo.
    Espera un número, lo multiplica y devuelve la respuesta.
    """
    # 1. RECV: Esperamos a recibir un mensaje
    # El proceso se bloquea aquí hasta que llegue algo
    mensaje_recibido = conexion.recv()
    print(f"🤖 Calculadora: Recibí el número {mensaje_recibido}")

    # Hacemos el cálculo
    resultado = mensaje_recibido * 10

    # 2. SEND: Enviamos la respuesta de vuelta
    conexion.send(resultado)
    print(f"🤖 Calculadora: Enviando resultado {resultado} y cerrando.")

    # 3. Cerramos nuestra parte de la conexión
    conexion.close()

if __name__ == "__main__":
    print("--- INICIO CONEXIÓN PIPE ---")

    # 1. Crear el Pipe
    # Retorna DOS objetos: uno para cada extremo del tubo.
    # conexion_padre: La usamos aquí en el main.
    # conexion_hijo: Se la pasamos al otro proceso.
    conexion_padre, conexion_hijo = multiprocessing.Pipe()

    # 2. Lanzar el proceso
    p = multiprocessing.Process(target=proceso_calculadora, args=(conexion_hijo,))
    p.start()

    # 3. Hablar desde el proceso principal
    numero_a_enviar = 5
    print(f"👨‍💻 Main: Enviando el número {numero_a_enviar}...")
    conexion_padre.send(numero_a_enviar)

    # 4. Esperar respuesta
    # Al igual que queue.get(), esto espera hasta que haya respuesta
    respuesta = conexion_padre.recv()
    print(f"👨‍💻 Main: ¡Respuesta recibida! El resultado es {respuesta}")

    p.join()
    print("--- FIN CONEXIÓN ---")

# Open with

## Escritura ('w') y Añadido ('a')

In [ ]:
def escribir_archivo():
    print("--- ESCRIBIENDO ARCHIVO ---")

    nombre_archivo = "diario.txt"

    # 1. Modo 'w': Crea el archivo (o lo blanquea si existe)
    # encoding='utf-8' es vital para acentos y ñ
    with open(nombre_archivo, "w", encoding="utf-8") as archivo:
        archivo.write("Día 1: Empezamos a aprender Python.\n")
        archivo.write("Hoy vimos diccionarios.\n")
        # Al salir de este bloque, Python hace archivo.close() automáticamente

    print("Escritura inicial terminada.")

    # 2. Modo 'a': Añadimos líneas sin borrar lo anterior
    with open(nombre_archivo, "a", encoding="utf-8") as archivo:
        archivo.write("Día 2: Hoy vimos Multiprocessing.\n")
        archivo.write("Espero no olvidar cerrar los procesos.\n")

    print("Actualización terminada.\n")

if __name__ == "__main__":
    escribir_archivo()

## Lectura Segura ('r')

In [ ]:
def leer_archivo():
    print("--- LEYENDO ARCHIVO ---")
    nombre_archivo = "diario.txt"

    # Aseguramos que exista para no tener error
    try:
        with open(nombre_archivo, "r", encoding="utf-8") as f:

            # FORMA A: .read() -> Lee TODO el archivo a una sola variable string.
            # Útil para archivos pequeños. Peligroso para archivos de gigabytes.
            contenido_completo = f.read()
            print("--- Contenido Completo ---")
            print(contenido_completo)

            # Para volver a leer, hay que "rebobinar" el puntero al inicio
            f.seek(0)

            # FORMA B: .readlines() -> Crea una LISTA de líneas
            print("--- Lista de líneas ---")
            lista_lineas = f.readlines()
            print(lista_lineas) # ['Dia 1...\n', 'Hoy vimos...\n']

            f.seek(0)

            # FORMA C: Iterar sobre el objeto archivo (LA MÁS EFICIENTE)
            # Esto lee línea por línea, no carga todo en memoria RAM a la vez.
            print("--- Lectura Línea a Línea (Eficiente) ---")
            for linea in f:
                # .strip() elimina el salto de línea (\n) sobrante
                print(f"Leído: {linea.strip()}")

    except FileNotFoundError:
        print("¡El archivo no existe! Ejecuta el ejemplo de escritura primero.")

if __name__ == "__main__":
    leer_archivo()

## Integración: with open + Multiprocessing

In [ ]:
import multiprocessing
import time

def guardar_log(id_proceso, candado):
    for i in range(3):
        time.sleep(0.5) # Simulamos trabajo

        # ⚠️ ZONA CRÍTICA: Solo un proceso escribe a la vez
        with candado:
            # Usamos 'a' (append) para no borrar lo que escribió el otro
            with open("registro_procesos.txt", "a", encoding="utf-8") as f:
                f.write(f"Proceso {id_proceso}: Mensaje número {i}\n")
        # ⚠️ FIN ZONA CRÍTICA
        # El candado se libera y el archivo se cierra aquí

if __name__ == "__main__":
    print("--- ESCRITURA CONCURRENTE ---")

    # Limpiamos el archivo antes de empezar
    with open("registro_procesos.txt", "w") as f:
        f.write("INICIO DEL LOG\n")

    candado = multiprocessing.Lock()
    procesos = []

    for i in range(4):
        p = multiprocessing.Process(target=guardar_log, args=(i, candado))
        procesos.append(p)
        p.start()

    for p in procesos:
        p.join()

    print("Todos los procesos han escrito en 'registro_procesos.txt' ordenadamente.")